# Import libraries

In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import joblib

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K

import warnings
warnings.filterwarnings('ignore')

print('Tensorflow version:', tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
print('Using cuda devices (GPU)' if gpus else 'GPU not available, using CPU')

In [ ]:
# Path and parameters
DATADIR = '/kaggle/input/cat-dog-images-for-classification/cat_dog'
H, W = 224, 224
EPOCHS = 50
BATCH_SIZE = 32
SEED = 42

# Data Preparation

In [ ]:
df = pd.read_csv('/kaggle/input/cat-dog-images-for-classification/cat_dog.csv')
df['filenames'] = df['image']
df['labels'] = df['labels'].map(lambda x: 'dog' if x == 1 else 'cat')
df.drop(columns=['image'], inplace=True)
df.head()

In [ ]:
df.labels.value_counts()

In [ ]:
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=SEED, stratify=df['labels'])
valid_df, test_df = train_test_split(temp_df, test_size=1/3, random_state=SEED, stratify=temp_df['labels'])
train_df.sample(frac=1, random_state=SEED)

train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print('Train size:', len(train_df))
print('Validation size:', len(valid_df))
print('Test size:', len(test_df))

train_df.to_csv('train.csv')
valid_df.to_csv('valid.csv')
test_df.to_csv('test.csv')

# Data Exploration

In [ ]:
dogs = list(df[df.labels == 'dog'].filenames)
cats = list(df[df.labels == 'cat'].filenames)

In [ ]:
def get_side(img, side_type, n=5):
    h, w, c = img.shape
    if side_type == 'horizontal':
        return np.ones((h, n, c))
    return np.ones((n, w, c))

def show_gallery(data, n, title):
    images = []
    vertical_images = []

    for i in range(n * n):
        img = load_img(os.path.join(DATADIR, data[i]), target_size=(W, H))
        img = img_to_array(img)
        hside = get_side(img, side_type='horizontal')
        images.append(img)
        images.append(hside)

        if (i + 1) % n == 0:
            himage = np.hstack((images))
            vside = get_side(himage, side_type="vertical")
            vertical_images.append(himage)
            vertical_images.append(vside)
            
            images = []

    gallery = np.vstack((vertical_images))
    plt.figure(figsize=(20, 20))
    plt.axis('off')
    plt.imshow(gallery.astype(np.uint8))
    plt.savefig(f'gallery_{title}.jpg', dpi=300)
    plt.show()

In [ ]:
show_gallery(dogs, n=10, title='dog')

In [ ]:
show_gallery(cats, n=10, title='cat')

# GradCAM & GuidedGradCAM class define

## GuidedBackprop

In [ ]:
@tf.custom_gradient
def guidedRelu(x):
    def grad(dy):
        return tf.cast(dy > 0, 'float32') * tf.cast(x > 0, 'float32') * dy
    return tf.nn.relu(x), grad

class GuidedBackprop:
    def __init__(self,model, layerName=None):
        self.model = model
        self.layerName = layerName
        self.gbModel = self.build_guided_model()
        
        if self.layerName == None:
            self.layerName = self.find_target_layer()

    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:
                return layer.name
        raise ValueError("Could not find 4D layer. Cannot apply Guided Backpropagation")

    def build_guided_model(self):
        gbModel = Model(
            inputs = [self.model.inputs],
            outputs = [self.model.get_layer(self.layerName).output]
        )
        layer_dict = [layer for layer in gbModel.layers[1:] if hasattr(layer, "activation")]
        for layer in layer_dict:
            if layer.activation == tf.keras.activations.relu:
                layer.activation = guidedRelu
        
        return gbModel
    
    def guided_backprop(self, images, upsample_size):
        """Guided Backpropagation method for visualizing input saliency."""
        with tf.GradientTape() as tape:
            inputs = tf.cast(images, tf.float32)
            tape.watch(inputs)
            outputs = self.gbModel(inputs)

        grads = tape.gradient(outputs, inputs)[0]

        saliency = cv2.resize(np.asarray(grads), upsample_size)

        return saliency

def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.25
    x = x.copy()
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.25

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

## GradCAM

In [ ]:
class GradCAM:
    def __init__(self, model, layerName=None):
        """
        model: pre-softmax layer (logit layer)
        """
        self.model = model
        self.layerName = layerName
            
        if self.layerName == None:
            self.layerName = self.find_target_layer()
    
    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:
                return layer.name
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM")
            
    def compute_heatmap(self, image, classIdx, upsample_size, eps=1e-5):
        gradModel = Model(
            inputs = [self.model.inputs],
            outputs = [self.model.get_layer(self.layerName).output, self.model.outputs]
        )
        # record operations for automatic differentiation
        
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOuts, preds) = gradModel(inputs) # preds after softmax
            loss = preds[0][:, classIdx]
        
        # compute gradients with automatic differentiation
        grads = tape.gradient(loss, convOuts)
        # discard batch
        convOuts = convOuts[0]
        grads = grads[0]
        norm_grads = tf.divide(grads, tf.reduce_mean(tf.square(grads)) + tf.constant(eps))
        
        # compute weights
        weights = tf.reduce_mean(norm_grads, axis=(0,1))
        cam = tf.reduce_sum(tf.multiply(weights, convOuts), axis=-1)
        
        # Apply reLU
        cam = np.maximum(cam, 0)
        cam = cam/np.max(cam)
        cam = cv2.resize(cam, upsample_size,interpolation=cv2.INTER_LINEAR)
        
        # convert to 3D
        cam3 = np.expand_dims(cam, axis=2)
        cam3 = np.tile(cam3, [1,1,3])
        
        return cam3
    
def overlay_gradCAM(img, cam3):
    cam3 = np.uint8(255 * cam3)
    cam3 = cv2.applyColorMap(cam3, cv2.COLORMAP_JET)
    
    new_img = 0.3 * cam3 + 0.5 * img
    
    return (new_img * 255.0 / new_img.max()).astype("uint8")

# Visualization function

In [ ]:
def show_gradCAMs(model, gradCAM, GuidedBP, im_ls, n, classes):
    """
    model: softmax layer
    """

    plt.subplots(figsize=(30, 10 * n))
    k = 1
    
    for i in range(n):
        img = cv2.imread(os.path.join(DATADIR, im_ls[i]))
        upsample_size = (img.shape[1], img.shape[0])

        # Show original image
        plt.subplot(n, 3, k)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(f'Filename: {im_ls[i]}', fontsize=20)
        plt.axis('off')

        # Show overlayed Grad
        plt.subplot(n, 3, k + 1)
        im = img_to_array(load_img(os.path.join(DATADIR, im_ls[i]), target_size=(W, H)))
        x = np.expand_dims(im, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x)
        idx = preds.argmax()

        cam3 = gradCAM.compute_heatmap(image=x, classIdx=idx, upsample_size=upsample_size)
        new_img = overlay_gradCAM(img, cam3)
        new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
        plt.imshow(new_img)
        plt.title(f'GradCAM - Pred: {classes[idx]}. Prob: {preds.max()}', fontsize=20)
        plt.axis('off')

        # Show Guided GradCAM
        plt.subplot(n, 3, k + 2)
        gb = GuidedBP.guided_backprop(x, upsample_size)
        guided_gradcam = deprocess_image(gb * cam3)
        guided_gradcam = cv2.cvtColor(guided_gradcam, cv2.COLOR_BGR2RGB)
        plt.imshow(guided_gradcam)
        plt.title("Guided GradCAM", fontsize=20)
        plt.axis("off")
        
        k += 3
        
    plt.show()

# Re-Train output layer of Resnet model on dogs and cats data

# Data generator

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    DATADIR,
    x_col='filenames',
    y_col='labels',
    target_size=(W, H),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

In [ ]:
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_generator = valid_datagen.flow_from_dataframe(
    valid_df,
    DATADIR,
    x_col='filenames',
    y_col='labels',
    target_size=(W, H),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    DATADIR,
    x_col='filenames',
    y_col='labels',
    target_size=(W, H),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Train Model

In [ ]:
resnet = ResNet50V2(weights='imagenet', pooling='avg', include_top=False)
for layer in resnet.layers:
    layer.trainable = False

fc1 = Dense(128)(resnet.layers[-1].output)
fc2 = Dense(128, name='dense_feature')(fc1)
dropout = Dropout(0.5)(fc2)
outputs = Dense(2, activation='softmax')(dropout)

model = Model(inputs=resnet.input, outputs=outputs)

In [ ]:
sgd = optimizers.SGD(learning_rate=1e-3, weight_decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10
)

In [ ]:
model.save('resnet50v2.h5')

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(121)
plt.plot(history.history['loss'], label='Train') 
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(122)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validtion')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.savefig('history.png')

plt.show()

# Evaluate Model

In [ ]:
model = load_model('/kaggle/input/models/tensorflow2/default/2/resnet50v2.h5')

test_loss, test_accuracy = model.evaluate(test_generator)

print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

# Observe GradCAM

In [ ]:
model_logit = Model(model.input, model.layers[-1].output)
fctrained_gradCAM = GradCAM(model_logit, layerName='conv5_block3_out')
fctrained_guidedBP = GuidedBackprop(model, layerName='conv5_block3_out')

In [ ]:
y_pred = model.predict(test_generator)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
results = test_df.copy()
results['predict'] = y_pred
true_dogs = list(results[(results.labels == 'dog') & (results.predict == 1)].filenames)
true_cats = list(results[(results.labels == 'cat') & (results.predict == 0)].filenames)
wrong_class = [x for x in results.filenames if x not in (true_dogs + true_cats)]

In [ ]:
len(wrong_class)

## Dogs

In [ ]:
show_gradCAMs(model, fctrained_gradCAM, fctrained_guidedBP, true_dogs, n=5, classes={0: 'cat', 1: 'dog'})

## Cats

In [ ]:
show_gradCAMs(model, fctrained_gradCAM, fctrained_guidedBP, true_cats, n=5, classes={0: 'cat', 1: 'dog'})

## Wrong classes

In [ ]:
show_gradCAMs(model, fctrained_gradCAM, fctrained_guidedBP, wrong_class, n=5, classes={0: 'cat', 1: 'dog'})

# Feature Extract (COLE Hamard Product - C-HP)

In [ ]:
class FeatureExtractor:
    def __init__(self, model, layerName=None):
        """
        model: pre-softmax layer (logit layer)
        """
        self.model = model
        self.layerName = layerName
            
        if self.layerName == None:
            self.layerName = self.find_target_layer()
    
    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:
                return layer.name
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM")
            
    def compute_c_hp(self, image, classIdx):
        gradModel = Model(
            inputs = [self.model.inputs],
            outputs = [self.model.get_layer(self.layerName).output, self.model.outputs]
        )
        
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOuts, preds) = gradModel(inputs) 
            loss = preds[0][:, classIdx]
        
        grads = tape.gradient(loss, convOuts)
        if grads is None:
            raise ValueError("Gradients is None")
        
        c_hp = (grads * convOuts)[0]
        
        c_hp_flat = tf.reshape(c_hp, [-1])
        c_hp_flat = tf.math.l2_normalize(c_hp_flat)
        return c_hp_flat.numpy()

def get_chp_on_dataframe(extractor, df):
    features = []
    labels = []
    for path in tqdm(df.filenames):

        img = img_to_array(load_img(os.path.join(DATADIR, path), target_size=(W, H)))
        x = np.expand_dims(img, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x, verbose=0)
        idx = preds.argmax()
        
        c_hp = extractor.compute_c_hp(image=x, classIdx=idx)
        features.append(c_hp)
        labels.append('dog' if idx == 1 else 'cat')

    return np.array(features), np.array(labels)

In [ ]:
extractor = FeatureExtractor(model, layerName='dense_feature')

In [ ]:
(X_train, y_train), y_train_true = get_chp_on_dataframe(extractor, train_df), train_df.labels
(X_test, y_test), y_test_true = get_chp_on_dataframe(extractor, test_df), test_df.labels

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print('Accuracy:', accuracy_score(y_test_true, y_pred)) # Accuracy with true labels
print('Agreement:', accuracy_score(y_test, y_pred)) # Fidelity

In [ ]:
joblib.dump(knn, 'knn_chp_norm_model.pkl')

In [ ]:
rand_idx = np.random.randint(0, len(test_df))
path = test_df.filenames[rand_idx]
# path = 'dog.6131.jpg'
classes = {0: 'cat', 1: 'dog'}

img = cv2.imread(os.path.join(DATADIR, path))
upsample_size = (img.shape[1], img.shape[0])

plt.subplots(figsize=(30, 20))

# Show original image
plt.subplot(231)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title(f'Filename: {path}', fontsize=20)
plt.axis('off')

# Show overlayed Grad
plt.subplot(232)
im = img_to_array(load_img(os.path.join(DATADIR, path), target_size=(W, H)))
x = np.expand_dims(im, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
idx = preds.argmax()

cam3 = fctrained_gradCAM.compute_heatmap(image=x, classIdx=idx, upsample_size=upsample_size)
new_img = overlay_gradCAM(img, cam3)
new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
plt.imshow(new_img)
plt.title(f'GradCAM - Pred: {classes[idx]}. Prob: {preds.max()}', fontsize=20)
plt.axis('off')

# Show Guided GradCAM
plt.subplot(233)
gb = fctrained_guidedBP.guided_backprop(x, upsample_size)
guided_gradcam = deprocess_image(gb * cam3)
guided_gradcam = cv2.cvtColor(guided_gradcam, cv2.COLOR_BGR2RGB)
plt.imshow(guided_gradcam)
plt.title("Guided GradCAM", fontsize=20)
plt.axis("off")

######################### Retrieved ##################################

c_hp = extractor.compute_c_hp(image=x, classIdx=idx)
knn_index = knn.kneighbors(c_hp.reshape(1, -1), n_neighbors=1, return_distance=False)[0, 0]
c_hp_retrieved = knn._fit_X[knn_index]
retrieved_path = train_df.filenames[knn_index]
retrieved_label = train_df.labels[knn_index]

img = cv2.imread(os.path.join(DATADIR, retrieved_path))
upsample_size = (img.shape[1], img.shape[0])

# Show retrieved image
plt.subplot(234)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title(f'Retrieved Filename: {retrieved_path}', fontsize=20)
plt.axis('off')

# Show overlayed Grad
plt.subplot(235)
im = img_to_array(load_img(os.path.join(DATADIR, retrieved_path), target_size=(W, H)))
x = np.expand_dims(im, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
idx = preds.argmax()

cam3 = fctrained_gradCAM.compute_heatmap(image=x, classIdx=idx, upsample_size=upsample_size)
new_img = overlay_gradCAM(img, cam3)
new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
plt.imshow(new_img)
plt.title(f'GradCAM - Pred: {classes[idx]}. Prob: {preds.max()}', fontsize=20)
plt.axis('off')

# Show Guided GradCAM
plt.subplot(236)
gb = fctrained_guidedBP.guided_backprop(x, upsample_size)
guided_gradcam = deprocess_image(gb * cam3)
guided_gradcam = cv2.cvtColor(guided_gradcam, cv2.COLOR_BGR2RGB)
plt.imshow(guided_gradcam)
plt.title("Guided GradCAM", fontsize=20)
plt.axis("off")

plt.show()